# Tutorial 3: Betz Limit

Now that we have ran a couple simple models using WISDEM, let's look at OpenMDAO2. OpenMDAO2 is the code that connects the various components of turbine models into a cohesive whole that can be optimized in systems engineering problems.

One of the key features of OpenMDAO2 is the concept of a *subsystem*. These connect together to define optimization problems.

A classic problem of wind energy engineering is the Betz limit. This is the theoretical limit of power that can be extracted from wind by an idealized rotor. While a full explanation is beyond the scope of this tutorial, it is explained in many places online and in textbooks. One such explanation is on Wikipedia [https://en.wikipedia.org/wiki/Betz%27s_law](https://en.wikipedia.org/wiki/Betz%27s_law) This tutorial is based on [Optimizing an Actuator Disk Model to Find Betz Limit for Wind Turbines](http://openmdao.org/twodocs/versions/latest/examples/betz_limit/betz.html) in the OpenMDAO 2 documentation.

## Problem setup

According to the Betz limit, the maximum power a turbine can extract from wind is:

$$ C_p = \frac{16}{27} \approx 0.593 $$

Where \\(C_p\\) is the coefficient of power.

We will compute this limit using OpenMDAO 2 and a model of an idealized rotor using an actuator disk.

Here is our actuator disc:

![actuator disc](img/actuator_disc.png)

Where \\(V_u\\) freestream air velocity, upstream of rotor, \\(V_r\\) is air velocity at rotor exit plane and \\(V_d\\) is slipstream air velocity downstream of rotor, all measured in \\(\frac{m}{s}\\).

There are few other variables we'll have:

- \\(a\\): Induced Velocity Factor
- *Area*: Rotor disc area in \\(m^2\\)
- *thrust*: Thrust produced by the rotor in N
- \\(C_t\\): Thrust coefficient
- *power*: Power produced by rotor in *W*
- \\(\rho\\): Air density in \\(kg /m^3\\)

Before we start in on the source code, let's look at a few key snippets of the code

## Declare your derivatives

We need to tell OpenMDAO which derivatives will need to be computed. That happens in the following lines:

```python
self.declare_partials('Vr', ['a', 'Vu'])
self.declare_partials('Vd', 'a')
self.declare_partials('Ct', 'a')
self.declare_partials('thrust', ['a', 'Area', 'rho', 'Vu'])
self.declare_partials('Cp', 'a')
self.declare_partials('power', ['a', 'Area', 'rho', 'Vu'])
```

Note that lines like `self.declare_partials('Vr', ['a', 'Vu'])` references both the derivatives \\(\partial V_r / \partial a\\) and \\(\partial V_r / \partial V_u\\).

The Jacobian in which we provide solutions to the derivatives is

```python
J['Vr', 'a'] = -Vu
J['Vr', 'Vu'] = one_minus_a
J['Vd', 'a'] = -2.0 * Vu
J['Ct', 'a'] = 4.0 - 8.0 * a
J['thrust', 'a'] = 0.5 * rho * Vu**2 * Area * J['Ct', 'a']
J['thrust', 'Area'] = 2.0 * Vu**2 * a * rho * one_minus_a
J['thrust', 'Vu'] = 4.0 * a_area_rho_vu * one_minus_a
J['Cp', 'a'] = 4.0 * a * (2.0 * a - 2.0) + 4.0 * one_minus_a**2
J['power', 'a'] = 2.0 * Area * Vu**3 * a * rho * (2.0 * a - 2.0) + 2.0 * Area * Vu**3 * rho * one_minus_a**2
J['power', 'Area'] = 2.0 * Vu**3 * a * rho * one_minus_a ** 2
J['power', 'rho'] = 2.0 * a_times_area * Vu ** 3 * (one_minus_a)**2
J['power', 'Vu'] = 6.0 * Area * Vu**2 * a * rho * one_minus_a**2
```

Now we can make an `ActuatorDisc` class that models the actuator disc for the optimization.

In [1]:
import openmdao.api as om

class ActuatorDisc(om.ExplicitComponent):
    def setup(self):
        # Inputs into the the model
        self.add_input('a', 0.5, desc='Indcued velocity factor')
        self.add_input('Area', 10.0, units='m**2', desc='Rotor disc area')
        self.add_input('rho', 1.225, units='kg/m**3', desc='Air density')
        self.add_input('Vu', 10.0, units='m/s', desc='Freestream air velocity, upstream of rotor')

        # Outputs
        self.add_output('Vr', 0.0, units='m/s', desc='Air velocity at rotor exit plane')
        self.add_output('Vd', 0.0, units='m/s', desc='Slipstream air velocity, downstream of rotor')
        self.add_output('Ct', 0.0, desc='Thrust coefficient')
        self.add_output('Cp', 0.0, desc='Power coefficient')
        self.add_output('power', 0.0, units='W', desc='Power produced by the rotor')
        self.add_output('thrust', 0.0, units='m/s')

        self.declare_partials('Vr', ['a', 'Vu'])
        self.declare_partials('Vd', 'a')
        self.declare_partials('Ct', 'a')
        self.declare_partials('thrust', ['a', 'Area', 'rho', 'Vu'])
        self.declare_partials('Cp', 'a')
        self.declare_partials('power', ['a', 'Area', 'rho', 'Vu'])

    def compute(self, inputs, outputs):
        a = inputs['a']
        Vu = inputs['Vu']
        rho = inputs['rho']
        Area = inputs['Area']
        qA = 0.5 * rho * Area * Vu ** 2
        outputs['Vd'] = Vd = Vu * (1 - 2 * a)
        outputs['Vr'] = 0.5 * (Vu + Vd)
        outputs['Ct'] = Ct = 4 * a * (1 - a)
        outputs['thrust'] = Ct * qA
        outputs['Cp'] = Cp = Ct * (1 - a)
        outputs['power'] = Cp * qA * Vu
        
    def compute_partials(self, inputs, J):
        a = inputs['a']
        Vu = inputs['Vu']
        Area = inputs['Area']
        rho = inputs['rho']

        a_times_area = a * Area
        one_minus_a = 1.0 - a
        a_area_rho_vu = a_times_area * rho * Vu
 
        J['Vr', 'a'] = -Vu
        J['Vr', 'Vu'] = one_minus_a
        J['Vd', 'a'] = -2.0 * Vu
        J['Ct', 'a'] = 4.0 - 8.0 * a
        J['thrust', 'a'] = 0.5 * rho * Vu**2 * Area * J['Ct', 'a']
        J['thrust', 'Area'] = 2.0 * Vu**2 * a * rho * one_minus_a
        J['thrust', 'Vu'] = 4.0 * a_area_rho_vu * one_minus_a
        J['Cp', 'a'] = 4.0 * a * (2.0 * a - 2.0) + 4.0 * one_minus_a**2
        J['power', 'a'] = 2.0 * Area * Vu**3 * a * rho * (2.0 * a - 2.0) + 2.0 * Area * Vu**3 * rho * one_minus_a**2
        J['power', 'Area'] = 2.0 * Vu**3 * a * rho * one_minus_a ** 2
        J['power', 'rho'] = 2.0 * a_times_area * Vu ** 3 * (one_minus_a)**2
        J['power', 'Vu'] = 6.0 * Area * Vu**2 * a * rho * one_minus_a**2

### Now, we can add this as a subsystem to an optimization

First, we create a new problem to optimize.

In [2]:
prob = om.Problem()

Now make some independent input variables. Note that they are called "outputs" from the point of view of the rest of the problm, which gets the independent variables from the outputs of an `om.IndepVarComp` component. In essence, this is like defining a bunch of independent variables and publishing them to the rest of the problem.

In [3]:
indeps = prob.model.add_subsystem('indeps', om.IndepVarComp(), promotes=['*'])
indeps.add_output('a', 0.5)
indeps.add_output('Area', 10.0, units='m**2')
indeps.add_output('rho', 1.225, units='kg/m**3')
indeps.add_output('Vu', 10.0, units='m/s')

The ActuatorDisc model we created above needs the inputs specified in this line. OpenMDAO will detect the promoted OUTPUTS of 'indeps' and connect them to the promoted INPUTS of 'a_disk'.

In [5]:
prob.model.add_subsystem('a_disk', ActuatorDisc(), promotes_inputs=['a', 'Area', 'rho', 'Vu'])

Next, we set up an optimizer:

In [6]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.model.add_design_var('a', lower=0.0, upper=1.0)
prob.model.add_design_var('Area', lower=0.0, upper=1.0)

We want to maximize the objective, but OpenMDAO will want to minimize it. So we minimize the negative to find the maximum. Note that `Cp` is not promoted from `a_disk`. Therefore we must reference it with `a_disk.Cp`

In [7]:
prob.model.add_objective('a_disk.Cp', scaler=-1.0)

Now we can run the optimization:

In [8]:
prob.setup()
prob.run_driver()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.5925925906659251
            Iterations: 5
            Function evaluations: 6
            Gradient evaluations: 5
Optimization Complete
-----------------------------------


False

Above, we see a summary of the steps in our optimization. Don't worry about the output `False` for now. Next, we list the inputs and outputs that are problem used. Look at the outputs that are defined from our component with our independent variables and our `compute()` method in the class `ActuatorDisc`:

In [12]:
prob.model.list_inputs(values=False, hierarchical=False)
prob.model.list_outputs(values=False, hierarchical=False)

4 Input(s) in 'model'
---------------------

varname    
-----------
a_disk.a   
a_disk.Area
a_disk.rho 
a_disk.Vu  


10 Explicit Output(s) in 'model'
--------------------------------

varname      
-------------
indeps.a     
indeps.Area  
indeps.rho   
indeps.Vu    
a_disk.Vr    
a_disk.Vd    
a_disk.Ct    
a_disk.Cp    
a_disk.power 
a_disk.thrust


0 Implicit Output(s) in 'model'
-------------------------------



[('indeps.a', {}),
 ('indeps.Area', {}),
 ('indeps.rho', {}),
 ('indeps.Vu', {}),
 ('a_disk.Vr', {}),
 ('a_disk.Vd', {}),
 ('a_disk.Ct', {}),
 ('a_disk.Cp', {}),
 ('a_disk.power', {}),
 ('a_disk.thrust', {})]

## Finally, the result:

In [13]:
print(f"Coefficient of power Cp = {prob['a_disk.Cp']}")
print(f"Induction factor a={prob['a']}")
print(f"Rotor disc Area={prob['Area']} m^2")

Coefficient of power Cp = [0.59259259]
Induction factor a=[0.33335528]
Rotor disc Area=[1.] m^2


And there we have it. This matched the Betz limit of:

$$ C_p = \frac{16}{27} \approx 0.593 $$